In [11]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sns
import geopandas as gpd

import cartopy.crs as ccrs
import cartopy.feature as cf
import cartopy.feature as cfeature

import scipy.stats as stats

from matplotlib import patches
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec

import shapely.geometry as sg
from shapely.geometry import Point

import sys
sys.path.append(os.path.abspath(".."))
from function import DOWN_raw

import warnings
warnings.filterwarnings('ignore')

## Export Weibull parameters and percent of data per year

In [12]:
QC_method = 'QCv1_Flag1'
# QC_method = 'QCv1_Flag_Rec'

In [13]:
yy_s, yy_e = 2002, 2023
years_num = yy_e - yy_s + 1

In [14]:
lon_min, lon_max, lat_min, lat_max, area, toll = 6.5, 19, 36.5, 48, 'ITALY', 0.002

In [15]:
veneto_dir = os.path.join('/','media','arturo','T9','Data','shapes','Europa','Italy')

if os.path.exists(veneto_dir):
    REGIONS = gpd.read_file(os.path.join(veneto_dir,'Italy_regions.geojson'))
else:
    raise SystemExit(f"File not found: {veneto_dir}")

In [16]:
obs_base = os.path.join('/','media','arturo','T9','Data','Italy', 'stations', 'Italy_raingauges', 'QC_MIX')

In [17]:
METADATA = pd.read_csv(os.path.join(obs_base, 'METADATA', 'METADATA_INTENSE_UPDATE.csv'))
METADATA["Lat"] = np.round(METADATA["Lat"], 6)
METADATA["Lon"] = np.round(METADATA["Lon"], 6)
METADATA.head(2)

,Unnamed: 0,File,ISO,Region,Code,Name,Elevation,Or_EPSG,CRS_E,CRS_N,...,Source,TimeZon,Elevation.1,Orig_EPSG,Year_Start,Year_End,INTENSE_Resp,TimeZone,elevation_DEM,Elevation_update
0,0,IT-322_AA_6535_0060_QCv1.csv,IT-322,Alto-Adige,AA_6535,VANDOIESDISOPRA,746.0,32632.0,710057.0,5187849.0,...,Alto-Adige,UTC,NaN,NaN,NaN,NaN,NaN,NaN,745.9,746.0
1,1,IT-322_AA_5650_0060_QCv1.csv,IT-322,Alto-Adige,AA_5650,SELVADEIMOLINI,1141.0,32632.0,718972.0,5196967.0,...,Alto-Adige,UTC,NaN,NaN,NaN,NaN,NaN,NaN,1140.0,1141.0


In [18]:
# dir_read = os.path.join(dir_base, 'data_1h')
dir_read = os.path.join(obs_base, 'data_1dy', 'data')
regions_ISO = [d for d in os.listdir(dir_read) if os.path.isdir(os.path.join(dir_read, d))]

In [ ]:
for nn in range(len(regions_ISO)):
    # print(f'Directory: {regions_ISO[nn]}')
    dir_tmp = os.path.join(obs_base, 'DATA_1dy', 'statistics', QC_method, regions_ISO[nn])
    if not os.path.exists(dir_tmp):
        os.makedirs(dir_tmp)

Directory: IT-210
Directory: IT-450
Directory: IT-230
Directory: IT-250
Directory: IT-321
Directory: IT-322
Directory: IT-340
Directory: IT-360
Directory: IT-420
Directory: IT-520
Directory: IT-550
Directory: IT-570
Directory: IT-620
Directory: IT-670
Directory: IT-720
Directory: IT-750
Directory: IT-770
Directory: IT-780
Directory: IT-820
Directory: IT-880
Directory: IT-650


In [133]:
for rr in range(len(regions_ISO)):#len(regions_ISO)

    region_ = regions_ISO[rr] 
    METADATA_clear = METADATA[METADATA['ISO']==region_].reset_index(inplace=False)

    print(f'Region: {region_} has {len(METADATA_clear)} stations')

    names, lats, lons, Q5, Q10, Q20, Q50, Q100 = [], [], [], [], [], [], [], []

    for nn in range(len(METADATA_clear)):#len(METADATA_clear)

        filename = f'{METADATA_clear['File'].values[nn]}'
        lat_ref = METADATA_clear['Lat'].values[nn]
        lon_ref = METADATA_clear['Lon'].values[nn]

        RAIN_pd = pd.read_csv(os.path.join(obs_base, 'DATA_1dy', 'data', region_, filename))
        RAIN_pd['Datetime'] = pd.to_datetime(RAIN_pd['Datetime'].values)
        RAIN_pd['Year'] = RAIN_pd['Datetime'].dt.year

        ss, ee = RAIN_pd['Year'].min(), RAIN_pd['Year'].max()

        TIME_vector = pd.date_range(start=f'{ss}-01-01', end=f'{ee}-12-31', freq='1D')
        pd_TIME = pd.DataFrame({'Datetime': TIME_vector})
        pd_TIME['Year'] = pd_TIME['Datetime'].dt.year
        
        RAIN_full = pd.merge(pd_TIME, RAIN_pd, on='Datetime', how='left')

        try:
            nan_per_year = RAIN_full.groupby('Year_x')[QC_method].apply(lambda x: x.isna().sum())
            RAIN_xr = xr.DataArray(
                            RAIN_pd[QC_method].values,
                            coords={'time':RAIN_pd['Datetime'].values}, 
                            dims=('time'))

        except:
            nan_per_year = RAIN_full.groupby('Year_x')['pre'].apply(lambda x: x.isna().sum())
            RAIN_xr = xr.DataArray(
                            RAIN_pd['pre'].values,
                            coords={'time':RAIN_pd['Datetime'].values}, 
                            dims=('time'))

        nan_per_year = ((nan_per_year / 366) * 100).round(1)

        NCWy, YEARSy = DOWN_raw.fit_yearly_weibull(RAIN_xr,1)
        N = NCWy[:,0]
        C = NCWy[:,1]
        W = NCWy[:,2]
        Y = YEARSy

        STATISTIC = pd.DataFrame({'Year':Y.astype(int), 'N': N, 'C': C, 'W':W})
        STATISTIC['NaN'] = STATISTIC['Year'].map(nan_per_year)

        dir_out = os.path.join(obs_base, 'DATA_1dy', 'statistics', QC_method, region_, filename)
        STATISTIC.to_csv(dir_out, header=True, index=False)

Region: IT-210 has 38 stations
Region: IT-450 has 438 stations
Region: IT-230 has 69 stations
Region: IT-250 has 228 stations
Region: IT-321 has 97 stations
Region: IT-322 has 57 stations
Region: IT-340 has 213 stations
Region: IT-360 has 203 stations
Region: IT-420 has 153 stations
Region: IT-520 has 406 stations
Region: IT-550 has 65 stations
Region: IT-570 has 156 stations
Region: IT-620 has 213 stations
Region: IT-670 has 22 stations
Region: IT-720 has 153 stations
Region: IT-750 has 36 stations
Region: IT-770 has 37 stations
Region: IT-780 has 158 stations
Region: IT-820 has 106 stations
Region: IT-880 has 95 stations
Region: IT-650 has 38 stations
